In [1]:
from rockdaisy.nomenclator import Nomenclator

ModuleNotFoundError: No module named 'rockdaisy'

In [2]:
nomenclator_filepath = '../data/nomenclator.txt'
nomenclator = Nomenclator(nomenclator_filepath)


In [3]:
nomenclator

In [ ]:

# Lookup any species or synonym
record = nomenclator.lookup("Perityle canescens")

In [5]:
# Lookup an accepted name
nomenclator['Galinsogeopsis canescens']

{'species': 'Galinsogeopsis canescens',
 'authors': '(Everly) I.H. Lichter-Marck',
 'accepted_name': 'Galinsogeopsis canescens',
 'accepted_authors': '(Everly) I.H. Lichter-Marck',
 'relationship': 'accepted'}

In [6]:
# Lookup a synonym
nomenclator['Perityle canescens']

{'species': 'Perityle canescens',
 'authors': 'Everly',
 'accepted_name': 'Galinsogeopsis canescens',
 'accepted_authors': '(Everly) I.H. Lichter-Marck',
 'relationship': 'synonym'}

In [8]:
import re
from collections import defaultdict

def parse_nomenclator_nested(text):
    nested_dict = defaultdict(lambda: {"synonyms": [], "tentative": []})
    current_accepted = None

    lines = text.strip().splitlines()

    for line in lines:
        original = line
        line = line.rstrip().lstrip()
        if not line:
            continue

        indent = len(original) - len(original.lstrip())

        # Split by symbols while keeping them
        tokens = re.findall(r'[=+]?[^=+\n]+', line)

        for token in tokens:
            token = token.strip().rstrip(',')
            if not token:
                continue

            if token.startswith('='):
                name = token[1:].strip()
                if current_accepted:
                    nested_dict[current_accepted]["synonyms"].append(name)
            elif token.startswith('+'):
                name = token[1:].strip()
                if current_accepted:
                    nested_dict[current_accepted]["tentative"].append(name)
            else:
                # This is likely a new accepted name
                current_accepted = token
                if current_accepted not in nested_dict:
                    nested_dict[current_accepted] = {"synonyms": [], "tentative": []}

    return nested_dict


In [9]:
filepath = '../data/nomenclator.txt'

with open(filepath, 'r', encoding='utf-8') as f:
    text = f.read()

In [10]:
mapping = parse_nomenclator(text)

In [11]:
mapping

{'Alomia tenuifolia (Phil.) Benth. & Hook. ex Reiche': 'Perityle tenuifolius (Phil.) I.H. Lichter-Marck',
 'Amauria brandegeeana (Rose) Rydb.': 'Perityle brandegeeana Rose',
 'Amauria carterae A.M. Powell': 'Perityle carterae (A.M. Powell) I.H. Lichter-Marck',
 'Amauria rotundifolia Benth.': 'Perityle rotundifolia (Benth.) T.S. Brandegee',
 'Chlamysperma arenarioides Hook. & Arn.': 'galeana pratensis (Kunth) Rydb.',
 'Closia anthemoides Phil.': 'Perityle emoryi Torr.',
 'Closia cotula J. Remy': 'Perityle emoryi Torr.',
 'Closia elata Phil.': 'Perityle emoryi Torr.',
 'Closia foliosa Phil.': 'Perityle emoryi Torr.',
 'Closia viridis Phil.': 'Perityle emoryi Torr.',
 'Corellia montana A.M. Powell': 'Galinsogeopsis montana (A.M. Powell) I.H. Lichter-Marck',
 'Galeana hastata La Llave': 'galeana pratensis (Kunth) Rydb',
 '': 'Perityle rotundata (Rydb.) Shinners',
 'Galinsogeopsis pennellii (B.L. Turner) I.H. Lichter-Marck': 'Galinsogeopsis hofmeisteria (Rydb.) I.H. Lichter-Marck',
 'Perity